In [10]:
%matplotlib widget
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import matplotlib
import numpy as np
import matplotlib.pyplot as plt 
from models.label_structure import struct_string_map
struct_idx_map = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9, 11:10, 12:11, 13:12, 14:13, 15:14}


def func_mask(t1: torch.Tensor, t2: torch.Tensor): # get the mask for t2
    n1 = t1.shape[0]
    n2 = t2.shape[0]
    
    arr1 = t1.unsqueeze(1)  #(n1, 1)
    arr2 = t2.unsqueeze(0)  #(1, n2)
    arr1 = arr1.repeat(1, n2)   # (n1, n2)
    arr2 = arr2.repeat(n1, 1)  # (n1, n2)
    mask2d = arr1 == arr2 # (n1, n2)
    
    arr3 = torch.ones(n1, n2)
    arr3 = arr3.masked_fill(mask2d == False, 0)
    arr3 = torch.sum(arr3, 0)  # (n2,)
    
    mask1d = arr3 > 0  # (n2,) bool
    return mask1d 


def viz_2_maps(attn_score: torch.Tensor, multi_struct_label: torch.Tensor, selected_edges: list, null_node:bool, list_struct_idx:list, list_viz_struct=None, scale_map=False, turnoff_xyticks=False): 
    # t1, t2 are 2d tensors and must have the same shape
    # attn_score: [num_struct, num_selected_edges, num_selected_edges]
    # multi_struct_label: [num_struct, num_selected_edges, num_selected_edges]
    assert attn_score.shape[0] == multi_struct_label.shape[0] and attn_score.shape[1] == multi_struct_label.shape[1] and attn_score.shape[2] == multi_struct_label.shape[2]
    num_struct = attn_score.shape[0]
    num_viz_struct = num_struct if list_viz_struct is None else len(list_viz_struct)
    
    new_xtick= []
    new_ytick = []
    for i in range(len(selected_edges)): 
        edge_i = selected_edges[i].numpy()  # a tuple
        str_idx = "(" + str(edge_i[0]) + "," + str(edge_i[1]) + ")"
        new_xtick.append(str_idx)
        new_ytick.append(str_idx)
    if null_node:
        new_xtick.append("NULL")
    fontdict = {'fontsize': 25}
    fig, axs = plt.subplots(num_viz_struct, 2, figsize=(48, 36*num_viz_struct))
    if num_struct == 1:
        struct_idx = int(list_struct_idx[0])
        struct_idx_str = struct_string_map[struct_idx]
        attn_score_i = attn_score[0,:,:]
        struct_label_i = multi_struct_label[0,:,:]
        if scale_map: 
            max_i = max(torch.max(attn_score_i).item(), torch.max(struct_label_i).item()) 
            attn_score_i = attn_score_i * (1/max_i)
            struct_label_i = struct_label_i * (1/max_i)
            
        axs[0].matshow(attn_score_i.detach().numpy(), vmin=0.0, vmax=1.0)
        axs[0].set_title("attn score " + struct_idx_str, fontdict={'fontsize':40})
        if turnoff_xyticks:
            axs[0].set_xticks([])
            axs[0].set_yticks([])
        else: 
            axs[0].set_xticks(range(len(new_xtick)))
            axs[0].set_xticklabels(new_xtick, fontdict = fontdict)
            axs[0].set_yticks(range(len(new_ytick)))
            axs[0].set_yticklabels(new_ytick, fontdict=fontdict)
        
        im = axs[1].matshow(struct_label_i.detach().numpy(), vmin=0.0, vmax=1.0)
        axs[1].set_title("struct label " + struct_idx_str, fontdict={'fontsize':40})
        if turnoff_xyticks:
            axs[1].set_xticks([])
            axs[1].set_yticks([])
        else: 
            axs[1].set_xticks(range(len(new_xtick)))
            axs[1].set_xticklabels(new_xtick, fontdict=fontdict)
            axs[1].set_yticks(range(len(new_ytick)))
            axs[1].set_yticklabels(new_ytick, fontdict=fontdict)
    else: 
        if list_viz_struct is None:
            for i in range(num_struct):
                struct_idx = int(list_struct_idx[i])
                struct_idx_str = struct_string_map[struct_idx]
                # im.set_clim(0, 0.5)
                attn_score_i = attn_score[i,:,:]  # [num_selected_edges, num_selected_edges]
                struct_label_i = multi_struct_label[i,:,:]  # [num_selected_edges, num_selected_edges]
                if scale_map: 
                    max_i = max(torch.max(attn_score_i).item(), torch.max(struct_label_i).item()) 
                    attn_score_i = attn_score_i * (1/max_i)
                    struct_label_i = struct_label_i * (1/max_i)
                    
                axs[i][0].matshow(attn_score_i.detach().numpy(), vmin=0.0, vmax=1.0)
                axs[i][0].set_title("attn score " + struct_idx_str, fontdict={'fontsize':50})
                if turnoff_xyticks:
                    axs[i][0].set_xticks([])
                    axs[i][0].set_yticks([])
                else: 
                    axs[i][0].set_xticks(range(len(new_xtick)))
                    axs[i][0].set_xticklabels(new_xtick, fontdict = fontdict)
                    axs[i][0].set_yticks(range(len(new_ytick)))
                    axs[i][0].set_yticklabels(new_ytick, fontdict=fontdict)

                im = axs[i][1].matshow(struct_label_i.detach().numpy(), vmin=0.0, vmax=1.0)
                axs[i][1].set_title("struct label " + struct_idx_str, fontdict={'fontsize':50})
                if turnoff_xyticks:
                    axs[i][1].set_xticks([])
                    axs[i][1].set_yticks([])
                else: 
                    axs[i][1].set_xticks(range(len(new_xtick)))
                    axs[i][1].set_xticklabels(new_xtick, fontdict=fontdict)
                    axs[i][1].set_yticks(range(len(new_ytick)))
                    axs[i][1].set_yticklabels(new_ytick, fontdict=fontdict)
        else: 
            for i in range(len(list_viz_struct)): 
                struct_idx = list_viz_struct[i] 
                struct_idx_str = struct_string_map[struct_idx]
                struct_tensor_idx = struct_idx_map[struct_idx]
                
                attn_score_i = attn_score[struct_tensor_idx,:,:]
                struct_label_i = multi_struct_label[struct_tensor_idx,:,:]
                if scale_map: 
                    max_i = max(torch.max(attn_score_i).item(), torch.max(struct_label_i).item()) 
                    attn_score_i = attn_score_i * (1/max_i)
                    struct_label_i = struct_label_i * (1/max_i)
                    
                axs[i][0].matshow(attn_score_i.detach().numpy(), vmin=0.0, vmax=1.0)
                axs[i][0].set_title("attn score " + struct_idx_str, fontdict={'fontsize':50})
                if turnoff_xyticks:
                    axs[i][0].set_xticks([])
                    axs[i][0].set_yticks([])
                else: 
                    axs[i][0].set_xticks(range(len(new_xtick)))
                    axs[i][0].set_xticklabels(new_xtick, fontdict = fontdict)
                    axs[i][0].set_yticks(range(len(new_ytick)))
                    axs[i][0].set_yticklabels(new_ytick, fontdict=fontdict)

                im = axs[i][1].matshow(struct_label_i.detach().numpy(), vmin=0.0, vmax=1.0)
                axs[i][1].set_title("struct label " + struct_idx_str, fontdict={'fontsize':50})
                if turnoff_xyticks:
                    axs[i][1].set_xticks([])
                    axs[i][1].set_yticks([])
                else: 
                    axs[i][1].set_xticks(range(len(new_xtick)))
                    axs[i][1].set_xticklabels(new_xtick, fontdict=fontdict)
                    axs[i][1].set_yticks(range(len(new_ytick)))
                    axs[i][1].set_yticklabels(new_ytick, fontdict=fontdict)
            
    plt.subplots_adjust(wspace=0.2, hspace=0.1)
    cb = plt.colorbar(im, ax=axs.ravel().tolist(), orientation="horizontal")
    cb.ax.tick_params(labelsize=20)

    plt.show()


def visualization(directory, train_val_test_opt: str, null_node=True, bsize_idx = 0, max_n_nodes_visualize = 9, dict_info=None, list_viz_struct=None, scale_map=False, turnoff_xyticks=False):
    assert train_val_test_opt in ["Train", "Val", "Test"]
    local_directory = os.path.join(directory, train_val_test_opt)
    fw = dict_info['fw']
    device = dict_info['device'] 
    # '/Epoch_' + str(self.current_epoch) + '_device_' + device_str + '_forward_' + str(i) + '_attn.pt'
    attn_score_filename = "Device_" + str(device) +  "_forward_" + str(fw) + "_attn.pt"
    struct_label_filename = "Device_" + str(device) +  "_forward_" + str(fw) + "_struct.pt"
    info_filename = "Device_" + str(device) +  "_forward_" + str(fw) + "_info.pt"
    attn_score_filepath = os.path.join(local_directory, attn_score_filename)
    struct_label_filepath = os.path.join(local_directory, struct_label_filename)
    info_filepath = os.path.join(local_directory, info_filename)

    attn_score = torch.load(attn_score_filepath).cpu()  # [bsize, num_struct, |E|, |E|+1]
    multi_struct_label = torch.load(struct_label_filepath).cpu()  # [bsize, num_struct, |E|, |E|+1]
    graph_info_dict = torch.load(info_filepath)
    
    G_n_nodes = graph_info_dict['G_n_nodes'].cpu()  # tensor(list)
    G_n_edges12 = graph_info_dict['G_n_edges'].cpu()  # tensor(list)
    G_null_indices = graph_info_dict['G_null_indices'].cpu()  # tensor(Bsize, E+1, 2) = [32, 87, 2]
    G_list_struct_idx = graph_info_dict['list_struct_idx']
    
    g_n_nodes = G_n_nodes[bsize_idx]  # scalar, not counting null node
    g_n_edges12 = G_n_edges12[bsize_idx]  # scalar
    g_attn_score = attn_score[bsize_idx]  # [num_struct, E, E+1] = [86, 87]
    g_multi_struct_label = multi_struct_label[bsize_idx]  # [num_struct, E, E+1] = [86, 87]
    num_struct = g_attn_score.shape[0]
    
    g_indices = G_null_indices[bsize_idx, :g_n_edges12, :]  # (e,2) = (g_nedges12, 2) we don't want indices for null node
    num_nodes_viz = min(max_n_nodes_visualize, g_n_nodes)  # not + 1 to not account for null node 
    viz_node_idxs = torch.range(start=0, end = num_nodes_viz-1, dtype=torch.int)
    mask1 = func_mask(viz_node_idxs, g_indices[:,0])
    mask2 = func_mask(viz_node_idxs, g_indices[:,1])
    mask = torch.logical_and(mask1, mask2)  # (g_n_edges12,) bool
    selected_edges = [g_indices[i] for i in range(g_n_edges12) if mask[i] == True]
    selected_edge_indices = [i for i in range(g_n_edges12) if mask[i] == True]
    
    if null_node:
        null_edge_idx = g_n_edges12
        viz_attn_score = []
        viz_struct_label = []
        for i in range(num_struct): 
            viz_attn_score_i = g_attn_score[i,:,:][selected_edge_indices, :][:, selected_edge_indices + [null_edge_idx]]  # [num_selected_edges, num_selected_edges+1]
            viz_attn_score_i = viz_attn_score_i.unsqueeze(0)  # [1, num_selected_edges, num_selected_edges+1]
            viz_attn_score.append(viz_attn_score_i)
            
            viz_struct_label_i = g_multi_struct_label[i,:,:][selected_edge_indices, :][:, selected_edge_indices + [null_edge_idx]]
            viz_struct_label_i = viz_struct_label_i.unsqueeze(0)  # [1, num_selected_edges, num_selected_edges+1]
            viz_struct_label.append(viz_struct_label_i)
        viz_attn_score = torch.cat(viz_attn_score, dim=0)  # [num_struct, num_selected_edges, num_selected_edges+1]
        viz_struct_label = torch.cat(viz_struct_label, dim=0)  # [num_struct, num_selected_edges, num_selected_edges+1]
        assert viz_attn_score.shape[0] == viz_struct_label.shape[0] and viz_attn_score.shape[1] == viz_struct_label.shape[1] and viz_attn_score.shape[2] == viz_struct_label.shape[2]
    else:
        viz_attn_score = []
        viz_struct_label = []
        for i in range(num_struct): 
            viz_attn_score_i = g_attn_score[i,:,:][selected_edge_indices, :][:, selected_edge_indices]  # [num_selected_edges, num_selected_edges]
            viz_attn_score_i = viz_attn_score_i.unsqueeze(0)  # [1, num_selected_edges, num_selected_edges]
            viz_attn_score.append(viz_attn_score_i) 
            
            viz_struct_label_i = g_multi_struct_label[i,:,:][selected_edge_indices, :][:, selected_edge_indices] 
            viz_struct_label_i = viz_struct_label_i.unsqueeze(0)  # [1, num_selected_edges, num_selected_edges]
            viz_struct_label.append(viz_struct_label_i)
        viz_attn_score = torch.cat(viz_attn_score, dim=0)  # [num_struct, num_selected_edges, num_selected_edges]
        viz_struct_label = torch.cat(viz_struct_label, dim=0)  # [num_struct, num_selected_edges, num_selected_edges]
        assert viz_attn_score.shape[0] == viz_struct_label.shape[0] and viz_attn_score.shape[1] == viz_struct_label.shape[1] and viz_attn_score.shape[2] == viz_struct_label.shape[2]
        
    viz_2_maps(attn_score=viz_attn_score, multi_struct_label=viz_struct_label, selected_edges=selected_edges, null_node=null_node, list_struct_idx=G_list_struct_idx, list_viz_struct=list_viz_struct, scale_map=scale_map, turnoff_xyticks=turnoff_xyticks)


# Before running visualization, set the following parameters:
# - directory: the folder for saving the attention maps
# - train_val_test_opt: choose one from ["Train", "Test", "Val"]
# - max_n_nodes_visualize:  When we set max_n_nodes_visualize < number of graph nodes, then the program will make visualization for a subgraph containing max_n_nodes_visualize from the original graph
# - list_viz_struct: We have 15 structures of indices 1,2,...,15. Specify the set of structures you want to use for visualization. For mapping between structure and index, please refer to struct_string_map in models/label_structure.py
fw = 0
device = 0
dict_info = {'fw': fw, 'device': device}
directory = "exps/12/dense/rand-type/attn"
visualization(directory, train_val_test_opt="Train" , bsize_idx=0, max_n_nodes_visualize=4, null_node=False, dict_info=dict_info, list_viz_struct=[11,12,13,14,15], scale_map=True, turnoff_xyticks=False)

/tmp/ipykernel_1591/179995080.py:184: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  viz_node_idxs = torch.range(start=0, end = num_nodes_viz-1, dtype=torch.int)
